<a href="https://colab.research.google.com/github/ucfilho/ANN/blob/master/ANN_2022/Image_05/class_005_feature_extraction_image_003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import pandas as pd
import random
import zipfile
from matplotlib import pyplot as plt
import skimage.feature
from string import digits
import seaborn as sns
import skimage
from google.colab import files

In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
try:
  import glrlm
except:
  !pip install pip glrlm

In [4]:
try:
  import mahotas
except:
  !pip install mahotas

In [5]:
import glrlm
from glrlm import GLRLM

In [6]:
import mahotas

In [7]:
# check https://pypi.org/project/glrlm/

In [8]:
!gdown 1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX

Downloading...
From: https://drive.google.com/uc?id=1L4mdCDBgybKqr5Wo-kqT8HXcBJ9HXqSX
To: /content/trash_nov_22_2018.zip
100% 42.8M/42.8M [00:00<00:00, 261MB/s]


In [9]:
FILE='trash_nov_22_2018.zip'

In [10]:
archive = zipfile.ZipFile('trash_nov_22_2018.zip', 'r')
archive.extractall()

In [11]:
img_name=[]
with zipfile.ZipFile(FILE, "r") as f:
    for name in f.namelist():
        img_name.append(name)

In [12]:
Img_Size = 80
ref = 'jpg'
notref = 'met'

In [13]:
X= []  
Y =[]
n = len(img_name)
i = 0
for i in range(n):
  name = img_name[i]
  if(ref in name):
    Y.append(name)
    X.append(cv2.imread(name,0))
m = len(Y)
print(n,m)

2527 2527


In [14]:
def new_names(letter,names):
  lista = []
  for x in names:
    lista.append(x+letter)
  return lista

In [15]:
# glcm features

def properties_glcm(img,Img_Size):
  glcm =[]
  glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
  resized = cv2.resize(img, (Img_Size,Img_Size))
  m = skimage.exposure.rescale_intensity(resized, in_range='image', out_range='dtype')
  g = skimage.feature.greycomatrix(m, [1, 2], [0, np.pi/2], levels=256,normed=True, symmetric=True)
  for x in glcm_names:
    prop = skimage.feature.greycoprops(g, x)
    glcm.append(np.max(prop))
  
  return glcm

glcm_names =['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']


features =[]

#for i in range(len(img_name)):
for i in range(3):
  glcm = []
  glrlm_add = []
  img = X[i]
  glcm = properties_glcm(img,Img_Size)
  
  img = cv2.imread(img_name[i])
  
  green_img = img[:,:,1]
  values_green = properties_glcm(green_img,Img_Size)

  red_img = img[:,:,0]
  values_red = properties_glcm(red_img,Img_Size)


  blue_img = img[:,:,2]
  values_blue = properties_glcm(blue_img,Img_Size)


  total= [*glcm,*values_red, *values_green, *values_blue]

  features.append(total)

names = glcm_names + new_names('_r',glcm_names)
names = names + new_names('_g',glcm_names)
names = names + new_names('_b',glcm_names)

df_glcm = pd.DataFrame(features,columns=names)

In [16]:
Arq = "output_excel_glcm.xlsx"
df_glcm.to_excel(Arq)
files.download(Arq)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
df_glcm

,contrast,dissimilarity,homogeneity,energy,correlation,ASM,contrast_r,dissimilarity_r,homogeneity_r,energy_r,...,homogeneity_g,energy_g,correlation_g,ASM_g,contrast_b,dissimilarity_b,homogeneity_b,energy_b,correlation_b,ASM_b
0,206.881439,6.102652,0.343446,0.034536,0.969564,0.001193,261.070606,7.099949,0.307393,0.034844,...,0.338496,0.035026,0.968492,0.001227,184.833510,5.682955,0.358269,0.032721,0.972300,0.001071
1,167.128838,7.898232,0.258750,0.030549,0.995489,0.000933,166.029394,7.874545,0.257509,0.029724,...,0.257037,0.030699,0.995414,0.000942,162.046793,7.769015,0.259415,0.030471,0.995662,0.000928
2,585.697929,9.518990,0.344960,0.032758,0.940465,0.001073,452.975859,8.677475,0.342636,0.033257,...,0.343602,0.032901,0.937854,0.001082,657.670530,10.013611,0.337010,0.031738,0.950131,0.001007


In [18]:
# glrlm features

glrlm_names =['SRE','LRE','GLU','RLU','RPC']

app = GLRLM()

def glrlm_features(img,Img_Size,model=app):
  resized = cv2.resize(img, (Img_Size,Img_Size))
  glrlm_get = model.get_features(resized, 8)
  return glrlm_get.Features

features =[]

#for i in range(len(img_name)):
for i in range(3):

  glrlm_add = []
  img = X[i]
  glrlm_add = glrlm_features(img,Img_Size)
  
  img = cv2.imread(img_name[i])
  
  green_img = img[:,:,1]
  glrlm_green = glrlm_features(green_img,Img_Size)

  red_img = img[:,:,0]
  glrlm_red = glrlm_features(red_img,Img_Size)

  blue_img = img[:,:,2]
  glrlm_blue = glrlm_features(blue_img,Img_Size)

  total= [*glrlm_add, *glrlm_red, *glrlm_green, *glrlm_blue]

  features.append(total)


names = glrlm_names + new_names('_r',glrlm_names)
names = names + new_names('_g',glrlm_names)
names = names + new_names('_b',glrlm_names)

df_glrlm = pd.DataFrame(features,columns=names)

In [19]:
Arq = "output_excel_glrlm.xlsx"
df_glrlm.to_excel(Arq)
files.download(Arq)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
df_glrlm

,SRE,LRE,GLU,RLU,RPC,SRE_r,LRE_r,GLU_r,RLU_r,RPC_r,SRE_g,LRE_g,GLU_g,RLU_g,RPC_g,SRE_b,LRE_b,GLU_b,RLU_b,RPC_b
0,1.678,1382.729,2229.419,3459.465,12.303,1.667,1783.275,2506.805,3029.011,10.953,1.664,2133.061,2827.918,2620.917,9.710,1.724,1978.358,2770.329,2833.798,9.920
1,1.716,1505.775,2396.657,3381.269,11.585,1.600,2308.858,3036.883,2347.299,9.187,1.686,1420.130,2325.112,3425.916,11.977,1.650,2158.067,2855.685,2610.714,9.742
2,1.988,1203.374,2013.655,5126.984,13.664,1.952,1119.150,1983.664,5068.666,13.890,1.999,1241.638,2027.638,5154.621,13.585,2.007,746.900,1609.898,6605.360,17.062


In [24]:
# Local Binary Patterns features
# check https://mahotas.readthedocs.io/en/release-1.0/lbp.html

def properties_LBP(img,Img_Size):
  lbp_features =[]
 
  # adding gaussian filter
  img = mahotas.gaussian_filter(img, 4)
  
  # setting threshold
  threshed = (img > img.mean())
  
  # making is labelled image
  labeled, n = mahotas.label(threshed)
  
  resp = mahotas.features.lbp(labeled,200, 5)
  
  return resp


features =[]

#for i in range(len(img_name)):
for i in range(3):
  glcm = []
  glrlm_add = []
  img = X[i]
  LBP = properties_LBP(img,Img_Size)
  
  img = cv2.imread(img_name[i])
  
  green_img = img[:,:,1]
  values_green = properties_LBP(green_img,Img_Size)

  red_img = img[:,:,0]
  values_red = properties_LBP(red_img,Img_Size)


  blue_img = img[:,:,2]
  values_blue = properties_LBP(blue_img,Img_Size)


  total= [*LBP,*values_red, *values_green, *values_blue]

  features.append(total)

names = glcm_names + new_names('_r',glcm_names)
names = names + new_names('_g',glcm_names)
names = names + new_names('_b',glcm_names)

#df_glcm = pd.DataFrame(features,columns=names)
df_LBP = pd.DataFrame(features)
df_LBP.shape

(3, 32)

In [25]:
Arq = "output_excel_LBP.xlsx"
df_LBP.to_excel(Arq)
files.download(Arq)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
df_LBP

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,98507.0,72820.0,20929.0,0.0,4352.0,0.0,0.0,0.0,98715.0,73261.0,...,0.0,0.0,110805.0,60474.0,20202.0,0.0,5127.0,0.0,0.0,0.0
1,112047.0,49528.0,35033.0,0.0,0.0,0.0,0.0,0.0,110941.0,51552.0,...,0.0,0.0,111847.0,49488.0,35267.0,0.0,6.0,0.0,0.0,0.0
2,128064.0,35953.0,27690.0,781.0,4120.0,0.0,0.0,0.0,122667.0,42130.0,...,0.0,0.0,130756.0,32819.0,28654.0,834.0,3545.0,0.0,0.0,0.0
